# Trying openAI API, trying text stuff

In [10]:
import requests
import pandas as pd
import time

def download_book_bible_api(book_name, translation='web'):
    """
    Downloads all chapters and verses of a Bible book from bible-api.com.
    
    Args:
        book_name (str): Name of the book (e.g., 'John', 'Genesis').
        translation (str): Translation version (e.g., 'web', 'kjv', 'asv').

    Returns:
        list of dicts: Each dict contains metadata and verse text.
    """
    results = []
    chapter = 1

    while True:
        passage = f"{book_name}+{chapter}"
        print(f"{translation} {passage}...")
        url = f"https://bible-api.com/{passage}?translation={translation}"
        response = requests.get(url)
        time.sleep(0.25)

        if response.status_code == 404:
            break  # No more chapters
        elif response.status_code != 200:
            time.sleep(1)
            continue
#            raise Exception(f"Error fetching {translation} {passage}: {response.status_code}")

        data = response.json()
        verses = data.get('verses', [])
        if not verses:
            break

        for verse in verses:
            results.append({
                'translation': translation,
                'book': verse['book_name'],
                'chapter': verse['chapter'],
                'verse_number': verse['verse'],
                'passage': f"{verse['book_name']} {verse['chapter']}:{verse['verse']}",
                'text': verse['text'].strip()
            })

        chapter += 1

    book = pd.DataFrame(results)
    return book

In [11]:
#est_web = download_book_bible_api('est', 'web')
#est_kjv = download_book_bible_api('est', 'kjv')
gen_web = download_book_bible_api('gen', 'web')

web gen+1...
web gen+2...
web gen+3...
web gen+4...
web gen+5...
web gen+6...
web gen+7...
web gen+8...
web gen+9...
web gen+10...
web gen+11...
web gen+12...
web gen+13...
web gen+14...
web gen+15...
web gen+16...
web gen+16...
web gen+16...
web gen+16...
web gen+16...
web gen+16...
web gen+17...
web gen+18...
web gen+19...
web gen+20...
web gen+21...
web gen+22...
web gen+23...
web gen+24...
web gen+25...
web gen+26...
web gen+27...
web gen+28...
web gen+29...
web gen+30...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+31...
web gen+32...
web gen+33...
web gen+34...
web gen+35...
web gen+36...
web gen+37...
web gen+38...
web gen+39...
web gen+40...
web gen+41...
web gen+42...
web gen+43...
web gen+44...
web gen+45...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+46...
web gen+47...
web gen+48...
web gen+49...
w

In [12]:
#est_web.iloc[:5]
gen_web.iloc[:5]

,translation,book,chapter,verse_number,passage,text
0,web,Genesis,1,1,Genesis 1:1,"In the beginning, God created the heavens and ..."
1,web,Genesis,1,2,Genesis 1:2,The earth was formless and empty. Darkness was...
2,web,Genesis,1,3,Genesis 1:3,"God said, “Let there be light,” and there was ..."
3,web,Genesis,1,4,Genesis 1:4,"God saw the light, and saw that it was good. G..."
4,web,Genesis,1,5,Genesis 1:5,"God called the light “day”, and the darkness h..."


In [4]:
#est_kjv.iloc[:5]

## Check for differences in translation

## Bring in the LLM
* I am using ollama to run LLMs locally

In [13]:
#MODEL = "mistral"
MODEL = "gemma3:1b"

In [18]:
import ollama
import numpy as np
from tqdm import tqdm


def ask_question_about_texts(texts_and_ids, context, question, verbose=False):
    """
    Ask a question about example texts and get a single answer.

    Input:
    texts_and_ids: list of dicts. Each item represents an example text, with keys: 
        text_id: string. A unique identifier to each example.
        text: string. Tthe text itself.
    context: string. A bit of context to explain where every text example come from.
        Specific format: this string should complete the sentence "Each text example ..."
    question: string. The question you want to answer based on the examples provided.
    verbose: bool. Iff true, print the prompt being sent to the LLM.

    Output:
    answer: string. The answer to the question, based on the provided text examples.
    """
#    texts_lines = '\n'.join([f"  - {item['text_id']}. {item['text']}" for item in texts_and_ids])
    texts_lines = '\n'.join([f"  - {item['text']}" for item in texts_and_ids])
    prompt = f"""Help me analyze a few examples of text. I'll show you examples of text and ask a question about them.
Each text example {context}

Here are the text examples:
{texts_lines}

Based on these text examples,
{question}
    """

    if verbose:
        print(prompt)
        print("-"*40)
    messages = [{"role": "user", "content": prompt}]
    # Notice, I want to discover themes based on the text, not internet-knowledge. I don't want the LLM to "think" too hard.
    response = ollama.chat(model=MODEL, messages=messages, think=False, 
                           options={
                               'stop': ["\n"]
                           })
    answer = response['message']['content']
    if verbose:
        total_dur_sec = response['total_duration'] / 1e9
        print(f"[Response][{total_dur_sec} sec]: {answer}")
        print("-"*30)

    return answer

def pick_best_answer(text, context, question, answers, verbose=False):
    """
    Pick the best answer (from a menu) for the question about a piece of text.

    Input:
    text: string. The piece of text that you wish to analyze.
    context: string. A bit of context to explain where this text came from.
        Specific format: this string should complete the sentence "This text ..."
    question: string. A question about the text.
    answers: list of K strings. The possible answers to pick from.
    verbose: bool. Iff true, print the prompt sent to the LLM.

    Output:
    answer_id: int in [0, K-1]. The index of the selected answer.
    answer: string. The selected answer that best answers the question about the text.
    """
    K = len(answers)
    answers_lines = '\n'.join([f"  - {ai}. {ans}" for (ai, ans) in enumerate(answers)])
    prompt = f"""Help me analyze a piece of text. I'll ask you a question about it with a menu of possible answers and you'll pick the best answer.
This text {context}

Here is the piece of text:
{text}

{question}
Pick the best answer out of these {K} options (tell me the answer number):
{answers_lines}
    """
    response_schema = {
        "type": "integer",
        "enum": list(range(K))
    }

    if verbose:
        print(prompt)
        print("-"*40)
    messages = [{"role": "user", "content": prompt}]
    # Again, I don't want this model to think hard, just look at the text I'm providing:
    response = ollama.chat(model=MODEL, messages=messages, think=False, format=response_schema,
                           options={'temperature':0.3})
    answer_id = int(response['message']['content'])
    answer = answers[answer_id]
    answer = answer.strip()
    return (answer_id, answer)

def discover_answers_to_a_question(texts_and_ids, context, question, n_answers, verbose=False):
    """
    Analyze a collection of text examples, and discover various answers to a question about the texts.

    Input:
    texts_and_ids: list of dicts. Each item represents a single text example and has keys:
        text_id: string. Unique identifier for this piece of text.
        text: string. The text itself.
    context: string. A bit of context about the texts, explaining where they came from.
        Specific format: this string should complete the sentence "Each text example ..."
    question: string. The question you wish to ask about the text examples.
    n_answers: int. How many different answers do you want to discover?
    verbose: bool. Iff true, print intermediate messages.

    Output:
    answers: list of strings (length n_answers). The answers to the question that were discovered from the texts.
    """
    n_iter = 4
    batch_size = 30
    single_ans_batch_size = 6
    N = len(texts_and_ids)
    
    answers = None
    for iter in range(n_iter):
        print("-"*40)
        print(f"Iteration {iter} (out of 0...{n_iter-1})")
        inds = np.random.choice(N, batch_size, replace=False)
        batch = [texts_and_ids[ind] for ind in inds]

        # Assign each text example to one of the answers:
        answer_ids = [-1 for i in range(batch_size)]
        if answers:
            for i in tqdm(range(batch_size), desc=f"[Iteration {iter}] Assigning texts to answers"):
                (answer_id, answer) = pick_best_answer(batch[i]['text'], context, question, answers)
                answer_ids[i] = answer_id
        else:
            answer_ids = np.random.randint(0, n_answers, size=batch_size).tolist()
        
        # Update each answer, based on the texts that were assigned to it:
        answers = [None for ansi in range(n_answers)]
        ans_batch_sizes = [-1 for ansi in range(n_answers)]
        for ansi in tqdm(range(n_answers), desc=f"[Iteration {iter}] Updating answers"):
            ansi_inds = np.where(np.array(answer_ids)==ansi)[0].tolist()
            n_i = len(ansi_inds)
            ans_batch_sizes[ansi] = n_i
            if n_i > 0:
                if n_i > single_ans_batch_size:
                    ansi_inds = np.random.choice(ansi_inds, single_ans_batch_size, replace=False)
                ansi_batch = [batch[ind] for ind in ansi_inds]
            else:
                rand_inds = np.random.choice(N, single_ans_batch_size, replace=False)
                ansi_batch = [texts_and_ids[ind] for ind in rand_inds]
            new_answer = ask_question_about_texts(ansi_batch, context, question, verbose=verbose)
            answers[ansi] = new_answer
        
        answer_items = list(zip(answers, ans_batch_sizes))
        print(f"[Iteration {iter}]: {answer_items}")
    
    return answers


In [20]:
texts_and_ids = gen_web.rename(columns={"passage": "text_id"}, inplace=False)[['text_id', 'text']].to_dict(orient='records')
context = "is a verse from the book of Genesis."
#question = "What does god want from humans? Your response should be in a single line. Give a short answer (up to 8 words), followed by parenthesis with a short quote from the texts as an example (quote up to 8 words and use '...' if necessary)"
question = "What does god want from humans? Give a short answer (up to 8 words) in a single line."
n_answers = 4
answers = discover_answers_to_a_question(texts_and_ids, context, question, n_answers, verbose=False)
for ansi, ans in enumerate(answers):
    print(f"{ansi}) {ans}")

----------------------------------------
Iteration 0 (out of 0...3)


[Iteration 0] Updating answers: 100%|██████████| 4/4 [00:10<00:00,  2.72s/it]


[Iteration 0]: [('To be tested, ultimately to live.', 8), ('God desires humans to be His creations.', 9), ('To give life and be part of humanity.', 4), ('To inherit land and offspring God gives.', 9)]
----------------------------------------
Iteration 1 (out of 0...3)


[Iteration 1] Updating answers: 100%|██████████| 4/4 [00:08<00:00,  2.23s/it]


[Iteration 1]: [('To live and eat, receiving a blessing. ', 2), ('God wants humans to bless his work.', 1), ('To be fruitful and blessed.', 27), ('To create and prosper humankind.', 0)]
----------------------------------------
Iteration 2 (out of 0...3)


[Iteration 2] Updating answers: 100%|██████████| 4/4 [00:13<00:00,  3.35s/it]


[Iteration 2]: [('Provide a satisfying life.', 0), ('To inherit and become a godly family.', 0), ('He wants humanity to be on earth.', 30), ('Serve and obey Him greatly.', 0)]
----------------------------------------
Iteration 3 (out of 0...3)


[Iteration 3] Updating answers: 100%|██████████| 4/4 [00:09<00:00,  2.37s/it]

[Iteration 3]: [('Love, acceptance, and blessings.', 0), ('God wants a blessing and a descendants.', 2), ('God wants humans to be fruitful. ', 25), ('To serve and have children.', 3)]
0) Love, acceptance, and blessings.
1) God wants a blessing and a descendants.
2) God wants humans to be fruitful. 
3) To serve and have children.


In [35]:
mention_esther = est_web[est_web['text'].apply(lambda text:'esther' in text.lower())]
texts_and_ids = mention_esther.rename(columns={"passage": "text_id"}, inplace=False)[['text_id', 'text']].to_dict(orient='records')
context = "is a verse from a story."
question = "Look at the character Esther. How would you describe her personality in one word? Your answer should be in a single line - first the personality (single word) and then in parenthesis a quote from the text that demonstrates Esther as such a person."
n_answers = 5
answers = discover_answers_to_a_question(texts_and_ids, context, question, n_answers, verbose=False)
for ansi, ans in enumerate(answers):
    print(f"{ansi}) {ans}")

----------------------------------------
Iteration 0 (out of 0...3)


[Iteration 0] Updating answers: 100%|██████████| 5/5 [00:20<00:00,  4.17s/it]


[Iteration 0]: [('Recklessly determined (She speaks to Hathach regarding Mordecai’s requests, displaying a determined focus on her responsibility.)', 8), ('Fierce (She demonstrated fierce determination and defiance when she orchestrated the banquet.)', 4), ('Duty (Esther said, “If it seems good to the king, let the king and Haman come today to the banquet that I have prepared for him.”) ', 9), ('Courteous (She presented the letter with calm assurance and dignified resolve.)', 5), ('Loyal (Esther replied, “My petition and my request is this…” )', 4)]
----------------------------------------
Iteration 1 (out of 0...3)


[Iteration 1] Updating answers: 100%|██████████| 5/5 [00:22<00:00,  4.49s/it]


[Iteration 1]: [('Resolute (“Who is he, and where is he who dared presume in his heart to do so?”)', 1), ('Rebellious (Esther said, “An adversary and an enemy, even this wicked Haman!”)', 12), ('Resolve (Esther resolves to find a way for the people despite the dangers she faces) ', 16), ('Concerned (“If it pleases the king, let it be granted to the Jews who are in Shushan to do tomorrow also according to today’s decree, and let Haman’s ten sons be hanged on the gallows.")', 1), ('**Virtuos** (She bravely sought a solution despite the daunting situation.)', 0)]
----------------------------------------
Iteration 2 (out of 0...3)


[Iteration 2] Updating answers: 100%|██████████| 5/5 [00:20<00:00,  4.00s/it]


[Iteration 2]: [('Compassionate (She bravely pleaded for their freedom, displaying resilience and a genuine desire to protect others).', 0), ('Resolve (She is pragmatic and courageous.)', 0), ('Resilient (“An adversary and an enemy, even this wicked Haman!”)', 0), ('**Compassionate** (She was “loved more than all the women” due to her caring actions.) ', 0), ('Respectful (Esther quickly obtained favor in his sight)', 30)]
----------------------------------------
Iteration 3 (out of 0...3)


[Iteration 3] Updating answers: 100%|██████████| 5/5 [00:22<00:00,  4.48s/it]

[Iteration 3]: [('Protective (Esther was described as protective of her people through her actions.)', 0), ('Reverent (The text establishes her as a woman of high standing and reverence, shown through her approach to the king) ', 0), ('Resilient (She endured immense hardship and sought to protect her people.) ', 0), ('Kind (Esther’s words frequently show a gentle and compassionate spirit.)', 30), ('Courageful (She continues to offer her leadership, displaying a decisive resolve, as evident when she urges Esther to go to the king.)', 0)]
0) Protective (Esther was described as protective of her people through her actions.)
1) Reverent (The text establishes her as a woman of high standing and reverence, shown through her approach to the king) 
2) Resilient (She endured immense hardship and sought to protect her people.) 
3) Kind (Esther’s words frequently show a gentle and compassionate spirit.)
4) Courageful (She continues to offer her leadership, displaying a decisive resolve, as eviden

In [26]:
texts_and_ids = est_web.rename(columns={"passage": "text_id"}, inplace=False)[['text_id', 'text']].to_dict(orient='records')
context = "is a verse from a story."
#question = "Who is the main character? Don't write a full sentence; just give the name (up to 3 words)"
question = "What is the mood of the scene (up to 2 words)?"
n_answers = 5
answers = discover_answers_to_a_question(texts_and_ids, context, question, n_answers, verbose=False)
for ansi, ans in enumerate(answers):
    print(f"{ansi}) {ans}")

----------------------------------------
Iteration 0 (out of 0...3)


[Iteration 0] Updating answers: 100%|██████████| 5/5 [00:14<00:00,  2.86s/it]


[Iteration 0]: [('Serious', 7), ('Regal, solemn', 3), ('Dreadful', 7), ('Serene or solemn.', 7), ('Wrath / Anxiety', 6)]
----------------------------------------
Iteration 1 (out of 0...3)


[Iteration 1] Updating answers: 100%|██████████| 5/5 [00:15<00:00,  3.11s/it]


[Iteration 1]: [('Bleak. ', 0), ('Sadness/Warning', 0), ('Grim and apprehensive.', 4), ('Oppressive.', 25), ('Angry', 1)]
----------------------------------------
Iteration 2 (out of 0...3)


[Iteration 2] Updating answers: 100%|██████████| 5/5 [00:16<00:00,  3.28s/it]


[Iteration 2]: [('Dark', 0), ('Hopeful', 0), ('Sadness', 24), ('Sad/Fearful ', 5), ('Wrath.', 1)]
----------------------------------------
Iteration 3 (out of 0...3)


[Iteration 3] Updating answers: 100%|██████████| 5/5 [00:17<00:00,  3.53s/it]

[Iteration 3]: [('Cold', 0), ('Conflict', 0), ('Discordant. ', 27), ('Contemplative & Stern', 3), ('Bleakness', 0)]
0) Cold
1) Conflict
2) Discordant. 
3) Contemplative & Stern
4) Bleakness


In [10]:
prompt = """Help me analyze a few examples of text. I'll show you examples of text and ask a question about them.
Each text example is a verse from the book of Esther, translated to English.

Here are the text examples:
  - Esther 8:11. In those letters, the king granted the Jews who were in every city to gather themselves together, and to defend their life, to destroy, to kill, and to cause to perish, all the power of the people and province that would assault them, their little ones and women, and to plunder their possessions,
  - Esther 5:4. Esther said, “If it seems good to the king, let the king and Haman come today to the banquet that I have prepared for him.”
  - Esther 2:20. Esther had not yet made known her relatives nor her people, as Mordecai had commanded her; for Esther obeyed Mordecai, like she did when she was brought up by him.
  - Esther 6:1. On that night, the king couldn’t sleep. He commanded the book of records of the chronicles to be brought, and they were read to the king.

Based on these text examples,
Who is the main character? Don't write a full sentence; just give the name (up to 3 words)"""
messages = [{"role": "user", "content": prompt}]
response = ollama.chat(MODEL, messages=messages, think=False)
print(response)

model='mistral' created_at='2025-12-14T22:54:40.8698288Z' done=True done_reason='stop' total_duration=1846784800 load_duration=24342200 prompt_eval_count=301 prompt_eval_duration=565706600 eval_count=7 eval_duration=1250556400 message=Message(role='assistant', content=' Esther, Mordecai', thinking=None, images=None, tool_name=None, tool_calls=None) logprobs=None


In [36]:
batch = est_web.rename(columns={"passage": "text_id"}, inplace=False)[['text_id', 'text']].iloc[:10].to_dict(orient='records')
context = "is a verse from the book of Esther."
question = "Where is this taking place? (upto 10 words)"
answer = ask_question_about_texts(batch, context, question, verbose=True)
print(answer)

Help me analyze a few examples of text. I'll show you examples of text and ask a question about them.
Each text example is a verse from the book of Esther.

Here are the text examples:
  - Esther 1:1. Now in the days of Ahasuerus (this is Ahasuerus who reigned from India even to Ethiopia, over one hundred twenty-seven provinces),
  - Esther 1:2. in those days, when the King Ahasuerus sat on the throne of his kingdom, which was in Shushan the palace,
  - Esther 1:3. in the third year of his reign, he made a feast for all his princes and his servants; the power of Persia and Media, the nobles and princes of the provinces, being before him.
  - Esther 1:4. He displayed the riches of his glorious kingdom and the honor of his excellent majesty many days, even one hundred eighty days.
  - Esther 1:5. When these days were fulfilled, the king made a seven day feast for all the people who were present in Shushan the palace, both great and small, in the court of the garden of the king’s palace.


[{'text_id': 'Esther 1:1',
  'text': 'Now it came to pass in the days of Ahasuerus, (this\nis Ahasuerus which reigned, from India even unto Ethiopia,\nover an hundred and seven and twenty provinces:)'},
 {'text_id': 'Esther 1:2',
  'text': 'That in those days, when the king Ahasuerus sat on the throne of his kingdom, which\nwas in Shushan the palace,'},
 {'text_id': 'Esther 1:3',
  'text': 'In the third year of his reign, he made a feast unto all his princes and his servants; the power of Persia and Media, the nobles and princes of the provinces,\nbeing before him:'},
 {'text_id': 'Esther 1:4',
  'text': 'When he shewed the riches of his glorious kingdom and the honour of his excellent majesty many days,\neven an hundred and fourscore days.'},
 {'text_id': 'Esther 1:5',
  'text': 'And when these days were expired, the king made a feast unto all the people that were present in Shushan the palace, both unto great and small, seven days, in the court of the garden of the king’s palace;'},


In [7]:
help(ollama.chat)

Help on method chat in module ollama._client:

chat(
    model: str = '',
    messages: Optional[Sequence[Union[Mapping[str, Any], Message]]] = None,
    *,
    tools: Optional[Sequence[Union[Mapping[str, Any], Tool, Callable]]] = None,
    stream: bool = False,
    think: Optional[Union[bool, Literal['low', 'medium', 'high']]] = None,
    logprobs: Optional[bool] = None,
    top_logprobs: Optional[int] = None,
    format: Optional[Union[Literal['', 'json'], JsonSchemaValue]] = None,
    options: Optional[Union[Mapping[str, Any], Options]] = None,
    keep_alive: Optional[Union[float, str]] = None
) -> Union[ChatResponse, Iterator[ChatResponse]] method of ollama._client.Client instance
    Create a chat response using the requested model.

    Args:
      tools:
        A JSON schema as a dict, an Ollama Tool or a Python Function.
        Python functions need to follow Google style docstrings to be converted to an Ollama Tool.
        For more information, see: https://google.github.i

In [37]:
messages = [{"role": "user", "content": "From the garden of eden, describe the prominent characteristic of the serpent. Give a short answer (1-2 words)"}]
response_schema = {
    "type": "object",
    "properties": {
        "label": {
            "type": "string",
            "enum": ["pretty", "slimy", "naked", "subtle"]
        }
    },
    "required": ["label"]
}
response = ollama.chat("mistral", messages, think=False, logprobs=True, top_logprobs=20, format=response_schema)
print(response['message']['content'])
response['logprobs']

{ "label" : "slimy" }


[Logprob(token='{', logprob=-17.007966995239258, top_logprobs=[TokenLogprob(token=' C', logprob=-0.7147648334503174), TokenLogprob(token=' De', logprob=-1.1218626499176025), TokenLogprob(token=' Sl', logprob=-2.856114625930786), TokenLogprob(token=' Ven', logprob=-2.928539514541626), TokenLogprob(token=' W', logprob=-4.09712028503418), TokenLogprob(token=' Craft', logprob=-4.604372024536133), TokenLogprob(token=' Po', logprob=-5.018490791320801), TokenLogprob(token=' Sed', logprob=-5.438754081726074), TokenLogprob(token=' S', logprob=-5.474285125732422), TokenLogprob(token=' Sin', logprob=-5.4879045486450195), TokenLogprob(token=' The', logprob=-5.620206832885742), TokenLogprob(token=' Int', logprob=-6.034480094909668), TokenLogprob(token=' Wis', logprob=-6.075253486633301), TokenLogprob(token=' Dead', logprob=-6.391136169433594), TokenLogprob(token=' Sub', logprob=-6.663027763366699), TokenLogprob(token=' c', logprob=-6.827630043029785), TokenLogprob(token=' Sh', logprob=-7.0430355072

In [30]:
response['logprobs']

[Logprob(token=' C', logprob=-0.7147648334503174, top_logprobs=[TokenLogprob(token=' C', logprob=-0.7147648334503174), TokenLogprob(token=' De', logprob=-1.1218626499176025), TokenLogprob(token=' Sl', logprob=-2.856114625930786), TokenLogprob(token=' Ven', logprob=-2.928539514541626), TokenLogprob(token=' W', logprob=-4.09712028503418), TokenLogprob(token=' Craft', logprob=-4.604372024536133), TokenLogprob(token=' Po', logprob=-5.018490791320801), TokenLogprob(token=' Sed', logprob=-5.438754081726074), TokenLogprob(token=' S', logprob=-5.474285125732422), TokenLogprob(token=' Sin', logprob=-5.4879045486450195), TokenLogprob(token=' The', logprob=-5.620206832885742), TokenLogprob(token=' Int', logprob=-6.034480094909668), TokenLogprob(token=' Wis', logprob=-6.075253486633301), TokenLogprob(token=' Dead', logprob=-6.391136169433594), TokenLogprob(token=' Sub', logprob=-6.663027763366699), TokenLogprob(token=' c', logprob=-6.827630043029785), TokenLogprob(token=' Sh', logprob=-7.043035507

In [29]:
print(response)

model='mistral' created_at='2025-12-11T15:11:08.3660262Z' done=True done_reason='stop' total_duration=1416230600 load_duration=25306200 prompt_eval_count=30 prompt_eval_duration=193859100 eval_count=7 eval_duration=1153837400 message=Message(role='assistant', content=' Cunning/Deceptive', thinking=None, images=None, tool_name=None, tool_calls=None) logprobs=[Logprob(token=' C', logprob=-0.7147648334503174, top_logprobs=[TokenLogprob(token=' C', logprob=-0.7147648334503174), TokenLogprob(token=' De', logprob=-1.1218626499176025), TokenLogprob(token=' Sl', logprob=-2.856114625930786), TokenLogprob(token=' Ven', logprob=-2.928539514541626), TokenLogprob(token=' W', logprob=-4.09712028503418), TokenLogprob(token=' Craft', logprob=-4.604372024536133), TokenLogprob(token=' Po', logprob=-5.018490791320801), TokenLogprob(token=' Sed', logprob=-5.438754081726074), TokenLogprob(token=' S', logprob=-5.474285125732422), TokenLogprob(token=' Sin', logprob=-5.4879045486450195), TokenLogprob(token=' 

In [6]:
resp = ask_a_local_ollama_llm("mistral", "What quality is special about the serpent in teh garden of eden story. Give a short answer (upto 3 words)")
print(resp)

model='mistral' created_at='2025-12-11T14:58:08.7489292Z' done=True done_reason='stop' total_duration=2947506300 load_duration=23307200 prompt_eval_count=32 prompt_eval_duration=205162000 eval_count=16 eval_duration=2656860300 message=Message(role='assistant', content=' Cunning or crafty, as per Genesis 3:1.', thinking=None, images=None, tool_name=None, tool_calls=None) logprobs=None


In [3]:
resp

'{"model":"mistral","created_at":"2025-12-11T14:52:14.0164362Z","response":" C","done":false}{"model":"mistral","created_at":"2025-12-11T14:52:14.2224172Z","response":"unning","done":false}{"model":"mistral","created_at":"2025-12-11T14:52:14.434562Z","response":" intelligence","done":false}{"model":"mistral","created_at":"2025-12-11T14:52:14.6414693Z","response":".","done":false}{"model":"mistral","created_at":"2025-12-11T14:52:14.8404203Z","response":"","done":true,"done_reason":"stop","context":[3,29473,2592,4813,1117,3609,1452,1040,1969,22325,1065,1479,29484,9527,1070,2150,1037,3606,29491,16872,1032,3253,5140,1093,21181,29477,29473,29538,3853,29499,4,29473,1102,14628,11663,29491],"total_duration":9800681400,"load_duration":6851342100,"prompt_eval_count":32,"prompt_eval_duration":2113491200,"eval_count":5,"eval_duration":822041400}'